# Colab Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
"""
Change directory to where this file is located
"""
%cd '/content/drive/MyDrive/DLSP/Drna-master'

/content/drive/MyDrive/DLSP/Drna-master


## Data Loader

In [3]:
import torch
import torch.utils.data as data
from torchvision import transforms
from PIL import Image
import sys
from config import INPUT_SIZE

class GetLoader(data.Dataset):
    def __init__(self, data_root, data_list, train = None, train_procent = 0.8, transform=None):
        self.transform = transform
        self.data_root = data_root
       # print(self.data_root)
        f = open(data_list, 'r')
        data_list = f.readlines()
        f.close()

        self.n_data = len(data_list)

        self.img_paths = []
        self.labels = []
        self.intlabels = {"Food": 0, "Clothes": 1, "Institution": 2, "Accessories": 3, "Transportation": 4, "Electronic": 5, "Necessities": 6, "Cosmetic": 7, "Leisure": 8, "Medical": 9}
        for data in data_list:
            image_path = data[:-1]
            label = image_path.split('/')[0]
            self.img_paths.append(image_path)
            self.labels.append(label)
        

    def __getitem__(self, item):
        img_path, label= self.img_paths[item], self.labels[item]
        label = self.intlabels[label]
        img_path_full = self.data_root+img_path
        img = Image.open(img_path_full).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)

        return img, label

    def __len__(self):
        return self.n_data

class TransformData(data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.data[index]
        if self.transform:
            x = self.transform(x)
        return x, y
    
    def __len__(self):
        return len(self.data)

In [4]:
torch.cuda.is_available()

True

In [5]:
from tqdm import tqdm
from time import sleep

## Train RESNET50 fine-tuning all parameters (Large Images)

In [ ]:
import os
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.nn import DataParallel
from datetime import datetime
from torch.optim.lr_scheduler import MultiStepLR
from config import BATCH_SIZE, PROPOSAL_NUM, SAVE_FREQ, LR, WD, resume, save_dir, INPUT_SIZE, INPUT_SIZE_s, VAL_PROCENT, RESIZED_SIZE, RESIZED_SIZE_s
from torch import nn
from core import resnet as model
#from core import model_vgg as model
from core import data_loader
from core.utils import init_log, progress_bar
from PIL import Image
import numpy as np

BATCH_SIZE = 6
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
start_epoch = 1
save_dir = os.path.join(save_dir, datetime.now().strftime('%Y%m%d_%H%M%S'))
if os.path.exists(save_dir):
    raise NameError('model dir exists!')
os.makedirs(save_dir)
print(save_dir)
logging = init_log(save_dir)
_print = logging.info

################################################ DATA #############################

train_transform = transforms.Compose([
    transforms.Resize(RESIZED_SIZE, Image.BILINEAR),
    transforms.RandomCrop(INPUT_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))
    ])


test_transform = transforms.Compose([
    transforms.Resize(RESIZED_SIZE, Image.BILINEAR),
    transforms.CenterCrop(INPUT_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))
    ])




Full_train_dataset = GetLoader(data_root='./data/train/',
                    data_list='./data/List/train_images_root.txt',
                    transform=None)
train_nr = len(Full_train_dataset)
train_dataset, val_dataset = torch.utils.data.random_split(Full_train_dataset,
                                               [train_nr - int(np.ceil(train_nr*(VAL_PROCENT))), int(np.ceil(train_nr*(VAL_PROCENT)))])

train_dataset =  TransformData(train_dataset, transform = train_transform)
val_dataset =  TransformData(val_dataset, transform = test_transform)


test_dataset = GetLoader(data_root='./data/test/',
                data_list='./data/List/test_images_root.txt',
                transform=test_transform)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2, drop_last=False)

validationloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE,
                                          shuffle=False, num_workers=2, drop_last=False)


testloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2, drop_last=False)

print("Number of training data:", len(train_dataset))
print("Number of validation data:", len(val_dataset))
print("Number of test data:", len(test_dataset))

################################################ DATA #############################

# define model
net = torchvision.models.resnet50(pretrained=True)
#net = model.resnet152(pretrained=True)
# CHANING THE LAST FC-LAYER TO FIT 10 CLASSES
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 10)

#resume = "../models/20211123_010154/002.ckpt"
if resume:
    ckpt = torch.load(resume)
    net.load_state_dict(ckpt['net_state_dict'])
    start_epoch = ckpt['epoch'] + 1

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)
net = net.cuda()
net = DataParallel(net)

for epoch in range(start_epoch, 6):
    # begin training
    print('--' * 50)
    net.train()
    train_loss = 0
    train_correct = 0
    total = 0
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {epoch}")

            img, label = data[0].cuda(), data[1].cuda()
            batch_size = img.size(0)
            optimizer.zero_grad()
            outputs = net(img)
            loss = criterion(outputs,label)
            loss.backward()
            optimizer.step()
            # calculate accuracy
            _, predict = torch.max(outputs, 1)
            total += batch_size
            train_correct += torch.sum(predict.data == label.data)
            train_loss += loss.item() * batch_size
            tepoch.set_postfix(loss=train_loss/total, accuracy=100. * float(train_correct)/total)
            sleep(0.1)
    # Printing status after each epoch and perform validation    
    if epoch % SAVE_FREQ == 0:
        train_acc = float(train_correct) / total
        train_loss = train_loss / total

        print(
            'epoch:{} - train loss: {:.3f} and train acc: {:.3f} total sample: {}'.format(
                epoch,
                train_loss,
                train_acc,
                total))
            
	      # evaluate on test set
        test_loss = 0
        test_correct = 0
        total = 0
        net.eval()
        for i, data in enumerate(validationloader):
            img, label = data[0].cuda(), data[1].cuda()
            batch_size = img.size(0)
            
            outputs = net(img)
            loss = criterion(outputs,label)
            
            
            # calculate accuracy
            _, predict = torch.max(outputs, 1)
            total += batch_size
            test_correct += torch.sum(predict.data == label.data)
            test_loss += loss.item() * batch_size

        test_acc = float(test_correct) / total
        test_loss = test_loss / total
        print(
            'epoch:{} - test loss: {:.3f} and test acc: {:.3f} total sample: {}'.format(
                epoch,
                test_loss,
                test_acc,
                total))
        

	      # save model
        net_state_dict = net.module.state_dict()
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        torch.save({
            'epoch': epoch,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'test_loss': test_loss,
            'test_acc': test_acc,
            'net_state_dict': net_state_dict},
            os.path.join(save_dir, '%03d.ckpt' % epoch))

print('finishing training')

../models/20211129_024953


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Number of training data: 93566
Number of validation data: 23392
Number of test data: 50182


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

----------------------------------------------------------------------------------------------------


Epoch 1: 100%|██████████| 15595/15595 [3:24:48<00:00,  1.27batch/s, accuracy=36.8, loss=1.85]

epoch:1 - train loss: 1.846 and train acc: 0.368 total sample: 93566


## Train RESNET50 fine-tuning last layer (Small Images)


In [ ]:
import os
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.nn import DataParallel
from datetime import datetime
from torch.optim.lr_scheduler import MultiStepLR
from config import BATCH_SIZE, PROPOSAL_NUM, SAVE_FREQ, LR, WD, resume, save_dir, INPUT_SIZE, INPUT_SIZE_s, VAL_PROCENT, RESIZED_SIZE, RESIZED_SIZE_s
from torch import nn
#from core import resnet as model
#from core import model_vgg as model
#from core import data_loader
#from core.utils import init_log, progress_bar
from PIL import Image
import numpy as np

BATCH_SIZE = 2
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
start_epoch = 1
save_dir = os.path.join(save_dir, datetime.now().strftime('%Y%m%d_%H%M%S'))
if os.path.exists(save_dir):
    raise NameError('model dir exists!')
os.makedirs(save_dir)
print(save_dir)
#logging = init_log(save_dir)
#_print = logging.info

################################################ DATA #############################

train_transform = transforms.Compose([
    transforms.Resize(RESIZED_SIZE_s, Image.BILINEAR),
    transforms.RandomCrop(INPUT_SIZE_s),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))
    ])


test_transform = transforms.Compose([
    transforms.Resize(RESIZED_SIZE_s, Image.BILINEAR),
    transforms.CenterCrop(INPUT_SIZE_s),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))
    ])




Full_train_dataset = GetLoader(data_root='./data/train/',
                    data_list='./data/List/train_images_root.txt',
                    transform=None)
train_nr = len(Full_train_dataset)
train_dataset, val_dataset = torch.utils.data.random_split(Full_train_dataset,
                                               [train_nr - int(np.ceil(train_nr*(VAL_PROCENT))), int(np.ceil(train_nr*(VAL_PROCENT)))])

train_dataset =  TransformData(train_dataset, transform = train_transform)
val_dataset =  TransformData(val_dataset, transform = test_transform)


test_dataset = GetLoader(data_root='./data/test/',
                data_list='./data/List/test_images_root.txt',
                transform=test_transform)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2, drop_last=False)

validationloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE,
                                          shuffle=False, num_workers=2, drop_last=False)


testloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2, drop_last=False)

print("Number of training data:", len(train_dataset))
print("Number of validation data:", len(val_dataset))
print("Number of test data:", len(test_dataset))

################################################ DATA #############################

# define model
net = torchvision.models.resnet50(pretrained=True)
#net = model.resnet152(pretrained=True)
# CHANING THE LAST FC-LAYER TO FIT 10 CLASSES
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 10)

#resume = "../models/20211123_010154/002.ckpt"
if resume:
    ckpt = torch.load(resume)
    net.load_state_dict(ckpt['net_state_dict'])
    start_epoch = ckpt['epoch'] + 1

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr = 0.01, momentum=0.9, weight_decay=WD)
scheduler = MultiStepLR(optimizer, milestones=[8, 20], gamma=0.1)
net = net.cuda()
net = DataParallel(net)

for epoch in range(start_epoch, 20):
    # begin training
    print('--' * 50)
    net.train()
    train_loss = 0
    train_correct = 0
    total = 0
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:
            tepoch.set_description(f"Epoch {epoch}")

            img, label = data[0].cuda(), data[1].cuda()
            batch_size = img.size(0)
            optimizer.zero_grad()
            outputs = net(img)
            loss = criterion(outputs,label)
            loss.backward()
            optimizer.step()
            # calculate accuracy
            _, predict = torch.max(outputs, 1)
            total += batch_size
            train_correct += torch.sum(predict.data == label.data)
            train_loss += loss.item() * batch_size
            tepoch.set_postfix(loss=train_loss/total, accuracy=100. * float(train_correct)/total)
            sleep(0.1)
    scheduler.step()
    # Printing status after each epoch and perform validation    
    if epoch % SAVE_FREQ == 0:
        train_acc = float(train_correct) / total
        train_loss = train_loss / total

        print(
            'epoch:{} - train loss: {:.3f} and train acc: {:.3f} total sample: {}'.format(
                epoch,
                train_loss,
                train_acc,
                total))
            
	      # evaluate on test set
        test_loss = 0
        test_correct = 0
        total = 0
        net.eval()
        for i, data in enumerate(validationloader):
            img, label = data[0].cuda(), data[1].cuda()
            batch_size = img.size(0)
            
            outputs = net(img)
            loss = criterion(outputs,label)
            
            
            # calculate accuracy
            _, predict = torch.max(outputs, 1)
            total += batch_size
            test_correct += torch.sum(predict.data == label.data)
            test_loss += loss.item() * batch_size

        test_acc = float(test_correct) / total
        test_loss = test_loss / total
        print(
            'epoch:{} - test loss: {:.3f} and test acc: {:.3f} total sample: {}'.format(
                epoch,
                test_loss,
                test_acc,
                total))
        

	      # save model
        net_state_dict = net.module.state_dict()
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        torch.save({
            'epoch': epoch,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'test_loss': test_loss,
            'test_acc': test_acc,
            'net_state_dict': net_state_dict},
            os.path.join(save_dir, '%03d.ckpt' % epoch))

print('finishing training')

../models/20211130_061609
Number of training data: 93566
Number of validation data: 23392
Number of test data: 50182


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


----------------------------------------------------------------------------------------------------


Epoch 1: 100%|██████████| 46783/46783 [3:28:47<00:00,  3.73batch/s, accuracy=32.3, loss=2.02]

epoch:1 - train loss: 2.020 and train acc: 0.323 total sample: 93566
